<p style="font-weight:bold;"> <span style="font-size: 36px"> Contractual Service Margin and Loss Component Switch</span> </p>

<p style="font-weight:bold;"> <span style="font-size: 21px"> Impact of the switch logic to Financial Performance and Balance Sheet</span> </p>

In this case study we look at allocation of Technical Margin to Contractual Service Margin (CSM) and Loss Component (LC) and its impact on the Financial Performance and Balance Sheet. 
<br> In particular, we are going to consider two possible implementation for the **switch logic** which controls how the allocation of technical margin swithces from CSM to LC and viceversa throughout the Analysis of Change (AoC). 

In practice, we consider two hypothetical GICs with the same parameters, modelled cash flows, actuals and we will look at the results obtained applying two different implementation of the **switch logic** and its implication to Financial figures.

 - **MZ 1.1 Multiple Switches**: the allocation of the Technical Margin to either CSM or LC can happen at any AoC Step. This means multiple switches can be observed, whenever the Technical Margin changes sign.
 - **MZ 1.2 Single Switch**: the allocation of the Technical Margin to either CSM or LC can happen only once. This means only if there is an effective switch (i.e. the technical margin changes sign between the Beginning of Period (BoP) and End of Period (EoP)), a switch is observed.

We will look at the reports for the first quarter of year 2021 (2021 3).

# Data Import

In [0]:
#!import "CsmSwitchDataImport"

In [0]:
var reportingYear = 2021; 
var reportingMonth = 3;

# Fulfilment Cash Flow

We start by looking at the Fulfilment Cash Flows. Data for both cases are exactly the same.

In [0]:
var fulfillmentCashflows = ifrs17.FulfillmentCashflows;
fulfillmentCashflows.ReportingNode = "CH";
fulfillmentCashflows.ReportingPeriod = (reportingYear, reportingMonth);
fulfillmentCashflows.ColumnSlices = new string[]{"GroupOfContract","EstimateType"};
fulfillmentCashflows.DataFilter = null;
(await fulfillmentCashflows.ToReportAsync) with {Height = 750}

# Written Actuals

Aslo the Actuals for both cases are exactly the same.

In [0]:
var writtenActual = ifrs17.WrittenActuals;
writtenActual.ReportingNode = "CH";
writtenActual.ReportingPeriod = (reportingYear, reportingMonth);
writtenActual.ColumnSlices = new string[]{"GroupOfContract"};
writtenActual.DataFilter =  null;
(await writtenActual.ToReportAsync) with {Height = 400}

# LRC Technical Margin (TM)

In the Technical Margin report we present a unified view on the figures that are allocated to either Contractual Service Margin or to Loss Component. 
<br> The Analysis of Change is expanded with few more steps such as **Experience Adjustment** and **Amortization**.


A negative value for Technical Margin correspond to an Asset and thus allocated to the CSM. Conversely, a positive value of the Technical Margin represents a Liability, which is then allocated to Loss Component.  

An Analysis of Change report allows one to explain how the closing figure is obtained from the opening value and from each and every change to it captured in an individual step (Model Correction, Release, ...). At each AoC step, the Technical Margin is recognized as Asset or Liability by looking at the sign of the aggregation of all the figures of the Technical Margin starting from the Opening to a given AoC step.

Through the Analysis of Change steps it is possible that such aggregation of the Technical Margin oscillates from positive to negative and viceversa several times. How the Technical Margin figures are then allocated to CSM/LC is controlled by the implementation of the **switch logic**.

In [0]:
var technicalMargins = ifrs17.TechnicalMargins;
technicalMargins.ReportingNode = "CH";
technicalMargins.ReportingPeriod = (reportingYear, reportingMonth);
technicalMargins.ColumnSlices = new string[]{"GroupOfContract"};
technicalMargins.DataFilter = null;
(await technicalMargins.ToReportAsync) with {Height = 600}

# Contractual Service Margin / Loss Component / Loss Recovery Component

The Contractual Service Margin (CSM) / Loss Component (LC) / Loss Recovery Component (LR) [report](https://portal.systemorph.cloud/project/ifrs17/env/v1.0.0/Report/ReportScopes#technical-margin) are here shown side by side as the allocation to profit or loss is done at each step of the Analysis of Change. 

Here, we show the result of two different implementations of the **switch logic** for the two Group of Insurance contract we are considering. 
<br> Group of Insurance contract called "MZ1.1" is calculated with our standard methodology (**Many switches**) which allows to switch at every AoC Step with no limitation of amount of switches. 
<br> Group of Insurance contract called "MZ1.2" is calculated with a switch logic (**One switch**) that shows only one switch when the number of total switches is odd, none otherwise. 

Because only the New Business figures drive the technical margin to the Liability side, one can immediately realize the main difference between these two switch choices: 
<br> MZ1.2 does not recognize any Loss Component.

In this case the **One switch** logic does not show any switch as both opening and closing figures are assets. The New Business contributions are then forced on the CSM side of this report with a negative sign to reflect the fact that they are decreasing the asset. 

Contrarily, the **Many switches** approach allows the New Business to be recognized in the Loss component. This generates a discrepancy between the In Force (asset allocated to the CSM) and New Business (Liability allocated to the Loss Component). This is resolved in the Combined section of the report where both In Force and New Business figures are considered. The Combined Liability AoC Step takes care of adding the two contributions together cancelling the Liability contributions and allocating the equivalent amount to the CSM. 

In [0]:
var allocatedTechnicalMargins = ifrs17.AllocatedTechnicalMargins;
allocatedTechnicalMargins.ReportingNode = "CH";
allocatedTechnicalMargins.ReportingPeriod = (reportingYear, reportingMonth);
allocatedTechnicalMargins.ColumnSlices = new string[]{"GroupOfContract", "EstimateType"};
allocatedTechnicalMargins.DataFilter = null;
(await allocatedTechnicalMargins.ToReportAsync) with {Height = 700}

# LRC Actuarial

The [Actuarial Liability for Remaining Coverage](https://portal.systemorph.cloud/project/ifrs17/env/v1.0.0/Report/ReportScopes#lrc-actuarial-actuarial-liability-for-remaining-coverage) report shows figures from Fulfilment Cash flow discounted with current yield curve, and the allocated techinical margin. 

In the Balance Sheet view the two approaches can be distinguished only by slicing in the data by EstimateType and noticing that New Business section of the CSM for MZ1.1 is completely missing (as it is allocated to the LC). Note that the same closing balance is reached in both cases as the Combined Liability AoC Step in the CSM for MZ1.1 brings the New Business contribution to the CSM. 

In [0]:
var actuarialLrc = ifrs17.ActuarialLrc;
actuarialLrc.ReportingNode = "CH";
actuarialLrc.ReportingPeriod = (reportingYear, reportingMonth);
actuarialLrc.ColumnSlices = new string[]{"GroupOfContract", "EstimateType"};
actuarialLrc.DataFilter = null;
(await actuarialLrc.ToReportAsync) with {Height = 750}

# Financial Performance

The [Financial Performance](https://portal.systemorph.cloud/project/ifrs17/env/v1.0.0/Report/ReportScopes#ifrs-17-financial-performance) report discloses the Change in Estimate of the IFRS 17 balance sheet items ([LRC](#lrc) and [LIC](#lic)) and the relevant incurred cash flows (Premiums, Claims, Expenses, etc...) for the given period.

As expected, the *Total comprehensive Income* is the same in both scenarios. 
<br> One observes differences in the *Insurance Sevice Result* (ISE) and *Insurance Finance income/Expense* (IFIE) sections of 0.0326 and -0.0326 respectively. These differences cancel out to produce the same value for *Profit and Loss*. The *Other Comprehensive Income* section reports the same figure in both scenarios. 

Drilling down into the data granularity one can realize the source of the above mentioned differences. 
<br>  Let's start from the IFIE contribution (Financial contributions): 
- MZ1.1 esplicitly recognizes a contribution to loss: -0.0163 this is the interest accretion on the liability side of the techinical margin. Contrarily, MZ1.2 does not recognizes any loss. 
- MZ1.1 reports a bigger figure for CSM than MZ1.2 this is because in MZ1.2 the interest on the asset and on the liability are both allocated to CSM and reported here as summed up (they carry opposite sign - check [CSM report](#contractual-service-margin-loss-component-loss-recovery-component)).

Continuing with the ISE contribution, one sees that both *Insurance Revenue* and *Insurance service Expense* carries a difference of 0.0163 between the two scenarios. 
- The difference in *Insurance service Expense* is due to having explicitly allocated the New Business of the TM to Loss Component. In this item the financial step (such as IA) is excluded. Thus, because overall the Loss amount to 0 the net effect of summing all the movements but IA result in $-$IA. This term cancels out the LC contribution reported in the Financial section. 
- The difference in *Insurance Revenue* is produced by the CSM contribution to the *Non Financial LRC changes*. The MZ1.1 Group of Insurance Contract includes here the Combined Liability AoC Step which includes the contribution of the IA computed on the New Business (Liability).


Worth to mention is how the choice of splitting between financial and non-financial contribution all figures reported in the Financial Performance makes explicit many contributions which finally cancel out but contribute to a trasparent report. 
<br> We recognize that without this split on the Loss Component part the financial performance would not report any contribution to Loss. Just by splitting the CSM in financial and non-financial contributions would produce the same final figure without explicit reporting of the liability. 

In [0]:
var financialPerformance = ifrs17.FinancialPerformance;
financialPerformance.ReportingNode = "CH";
financialPerformance.ReportingPeriod = (reportingYear, reportingMonth);
financialPerformance.ColumnSlices = new string[]{"GroupOfContract"};
financialPerformance.DataFilter = null;
(await financialPerformance.ToReportAsync) with { Height = 900, GroupDefaultExpanded = 3}